In [1]:
import logging
import operator as op

import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as trans
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt

import vmdata
import trainlib
import more_trans
from classloc import squeezenet

%matplotlib inline

logging.basicConfig(level=logging.INFO)

In [2]:
root = vmdata.prepare_dataset_root(9, (8,0,0))
vdset = vmdata.VideoDataset(root,
                            transform=trans.Compose([
                                trans.ToTensor(),
                                trans.Normalize(*vmdata.get_normalization_stats(root))
                            ]))

In [3]:
dataloader = data.DataLoader(vdset, batch_size=32, shuffle=False)

In [4]:
net = squeezenet.SqueezeNet(2, (480, 704))

/home/kw/Documents/Work/dolphins-mc-su18/rt/lib/python3.5/site-packages/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/home/kw/Documents/Work/dolphins-mc-su18/rt/lib/python3.5/site-packages/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [5]:
inputs = next(iter(dataloader))

In [6]:
inputs.size()

torch.Size([32, 3, 480, 704])

In [7]:
with torch.no_grad():
    predictions = net(inputs)

In [8]:
predictions.size()

torch.Size([32, 2])

In [10]:
vdset.release_mmap()

# Training

In [2]:
root = vmdata.prepare_dataset_root(9, (8,0,0))
transform = trans.Compose([
    trans.ToTensor(),
    trans.Normalize(*vmdata.get_normalization_stats(root)),
])
labels = 'runs/CH09-08_00_00/global-presence.txt'
savedir = 'runs/CH09-08_00_00/global-presence.0/save'
statdir = 'runs/CH09-08_00_00/global-presence.0/stat'
loaderparams = {'num_workers': 1, 'batch_size': 64}
max_epochs = 300
device = torch.device('cpu')

In [3]:
net = squeezenet.SqueezeNet(2, (480, 704)).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(net.parameters())
checkpointsaver = trainlib.CheckpointSaver(net, savedir)
statsaver = trainlib.StatSaver(statdir)
with vmdata.VideoDataset(root, transform=transform) as vdset:
    lvdset = vmdata.LabelledVideoDataset(labels, vdset)
    def getloader(subsetind):
        return data.DataLoader(lvdset, sampler=data.SubsetRandomSampler(subsetind), **loaderparams)
    perm = np.random.permutation(np.arange(len(lvdset)))
    train_ind, test_ind = perm[:-len(perm)//10], perm[-len(perm)//10:]
    for epoch in range(max_epochs):
        logging.info('Beginning epoch {}/{}'.format(epoch+1, max_epochs))
        loaders = [('train', getloader(train_ind)), ('eval', getloader(test_ind))]
        stats = {'train_loss': [], 'eval_loss': [], 'train_acc': 0.0, 'eval_acc': 0.0}
        for mode, loader in loaders:
            logging.info('- Beginning {} mode'.format(mode))
            getattr(net, mode)()
            torch.set_grad_enabled(mode == 'train')
            running_correct = 0.0
            running_total = 0.0
            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                stats[mode + '_loss'].append(loss.detach().item())
                running_total += inputs.size(0)
                running_correct += torch.sum(torch.argmax(outputs.detach(), dim=1) == labels.detach())
                if mode == train:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
            stats[mode + '_acc'] = running_correct / running_total
        checkpointsaver(epoch)
        statsaver(epoch, **stats)
        logging.info('- Stats: train_loss_mean={} train_acc={} test_loss_mean={} test_acc={}'
                     .format(np.mean(stats['train_loss']), stats['train_acc'],
                             np.mean(stats['eval_loss']), stats['eval_acc']))
torch.set_grad_enabled(True)

/home/kw/Documents/Work/dolphins-mc-su18/rt/lib/python3.5/site-packages/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/home/kw/Documents/Work/dolphins-mc-su18/rt/lib/python3.5/site-packages/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


FileExistsError: [Errno 17] File exists: 'runs/CH09-08_00_00/global-presence.0/save'